In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchsummary import summary

In [2]:
transform = {
'train': transforms.Compose(
    [transforms.Resize(256),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]),
'test' : transforms.Compose(
    [transforms.Resize(224),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
}

batch_size = 8

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform['train'])
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform['test'])
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class ModelA(nn.Module):
    def __init__(self, in_feature):
        super().__init__()
        self.FC = nn.Sequential(
            nn.Linear(in_feature, 512),
            nn.ReLU(),
            # nn.BatchNorm2d(512),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            # nn.BatchNorm2d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            # nn.BatchNorm2d(128),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        x = self.FC(x)
        return x

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

modelA = models.resnet18(pretrained=True)
num_feature = modelA.fc.in_features
modelA.fc = ModelA(num_feature)
# summary(modelA, (3, 224, 224), device='cpu')

modelB = models.resnet50(pretrained=True)
num_feature = modelB.fc.in_features
modelB.fc = ModelA(num_feature)
# summary(modelB, (3, 224, 224), device='cpu')

In [6]:
from tqdm import tqdm
def train(model, criterion, optimizer, Epoch, Data, total):
    criterion = criterion.to(device)
    optimizer = optimizer
    model.to(device)
    model.train()
    
    for epoch in range(Epoch):
        running_loss = 0.0
        running_corrects = 0.0
        T = 0
        for idx, data in enumerate(tqdm(Data)):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data)
            T += labels.size(0)

            if idx % total == (total - 1):
                print('Epoch: [%d/%d, %d] loss: %.3f'%((epoch+1), Epoch, idx, (running_loss / T)))
                print('Training AC: %.3f %%'%(running_corrects / T * 100))
                running_loss = 0.0
                running_corrects = 0.0
                T = 0

In [7]:
criiterion = nn.CrossEntropyLoss()
lr = 3e-4
Epoch = 10
optimizer_A = optim.Adam(modelA.parameters(), lr=lr)
optimizer_B = optim.Adam(modelB.parameters(), lr=lr)

In [8]:
train(modelA, criterion=criiterion, optimizer=optimizer_A, Epoch=Epoch, Data=train_loader, total=1000)

 16%|█▌        | 1005/6250 [00:26<01:59, 44.03it/s]

8000
Epoch: [1/10, 999] loss: 0.203
Training AC: 40.500 %


 32%|███▏      | 2005/6250 [00:49<01:35, 44.34it/s]

8000
Epoch: [1/10, 1999] loss: 0.155
Training AC: 57.188 %


 48%|████▊     | 3005/6250 [01:11<01:13, 44.40it/s]

...

 80%|████████  | 5005/6250 [01:51<00:27, 46.04it/s]

8000
Epoch: [10/10, 4999] loss: 0.021
Training AC: 95.313 %


 96%|█████████▌| 6005/6250 [02:13<00:05, 44.77it/s]

8000
Epoch: [10/10, 5999] loss: 0.019
Training AC: 95.513 %


100%|██████████| 6250/6250 [02:18<00:00, 44.99it/s]


In [9]:
train(modelB, criterion=criiterion, optimizer=optimizer_B, Epoch=Epoch, Data=train_loader, total=int(len(train_loader)/batch_size))

 13%|█▎        | 782/6250 [00:40<04:31, 20.13it/s]

6248
Epoch: [1/10, 780] loss: 0.251
Training AC: 22.615 %


 25%|██▌       | 1563/6250 [01:19<03:53, 20.10it/s]

6248
Epoch: [1/10, 1561] loss: 0.223
Training AC: 31.706 %


 38%|███▊      | 2344/6250 [01:58<03:14, 20.09it/s]

...

6248
Epoch: [10/10, 4685] loss: 0.044
Training AC: 88.396 %


 88%|████████▊ | 5470/6250 [04:50<00:40, 19.41it/s]

6248
Epoch: [10/10, 5466] loss: 0.044
Training AC: 88.236 %


100%|██████████| 6250/6250 [05:30<00:00, 18.92it/s]

6248
Epoch: [10/10, 6247] loss: 0.045
Training AC: 87.772 %


In [11]:
torch.save(modelA.state_dict(), 'resnet18.pth')
torch.save(modelB.state_dict(), 'resnet50.pth')